In [1]:
!ls

README.md  app	dataset  experiment.ipynb  main.py  requirements.txt


In [2]:
import pandas as pd
from PIL import Image
import os

In [7]:
annotations_path = "./dataset/combined_annotations.csv"

In [8]:
annotations = pd.read_csv(annotations_path)

In [9]:
annotations.head()

,file_path,label,season_id,survey_id
0,./combined_dataset\20130718_210056.jpg,4,2,1
1,./combined_dataset\20130718_210056_001.jpg,4,2,1
2,./combined_dataset\20130718_210056_002.jpg,4,2,1
3,./combined_dataset\20130718_210110.jpg,4,2,1
4,./combined_dataset\20130718_210628.jpg,4,2,1


In [12]:
annotations['file_path'] = annotations['file_path'].str.replace('\\', '/', regex=False)


In [15]:
annotations['file_path'] = "./dataset/" + annotations['file_path'] 

In [10]:
def load_image(file_path):
    try:
        # Открыть изображение с использованием PIL
        with Image.open(file_path) as img:
            img = img.convert("RGB")  # Преобразование в формат RGB
            return img
    except Exception as e:
        print(f"Ошибка при загрузке изображения {file_path}: {e}")
        return None

In [16]:
for _, row in annotations.iterrows():
    file_path = row['file_path']
    label = row['label']  # Метка (идентификатор лисы)
    
    # Проверка наличия файла
    if os.path.exists(file_path):
        image = load_image(file_path)
        print(f"Загружено изображение {file_path} с меткой {label}")
    else:
        print(f"Файл не найден: {file_path}")

Загружено изображение ./dataset/./combined_dataset/20130718_210056.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210056_001.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210056_002.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210110.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210628.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210628_001.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210628_002.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210653.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210653_001.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_210653_002.jpg с меткой 4
Загружено изображение ./dataset/./combined_dataset/20130718_211936.jpg с меткой 3
Загружено изображение ./dataset/./combined_dataset/20130718_211936_001.jpg

KeyboardInterrupt: 

In [17]:
from torchvision import transforms
import torch

In [18]:
# Пример преобразований изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Изменение размера
    transforms.ToTensor(),         # Преобразование в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

In [19]:
data = []
labels = []

In [20]:
for _, row in annotations.iterrows():
    file_path = row['file_path']
    label = row['label']
    
    if os.path.exists(file_path):
        image = load_image(file_path)
        if image:
            image = transform(image)  # Применение преобразований
            data.append(image)
            labels.append(label)

In [21]:
data_tensor = torch.stack(data)
labels_tensor = torch.tensor(labels)

In [22]:
print(f"Размер данных: {data_tensor.shape}")
print(f"Размер меток: {labels_tensor.shape}")

Размер данных: torch.Size([12478, 3, 224, 224])
Размер меток: torch.Size([12478])


In [4]:
data_tensor_path = "data_tensor.pt"
labels_tensor_path = "labels_tensor.pt"

In [24]:
torch.save(data_tensor, data_tensor_path)
torch.save(labels_tensor, labels_tensor_path)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset

In [5]:
data_tensor = torch.load(data_tensor_path)
labels_tensor = torch.load(labels_tensor_path)

/tmp/ipykernel_172532/1818967108.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_tensor = torch.load(data_tensor_path)
/tmp/ipykernel_172532/1818967108.py:2: Future

In [7]:
print(f"Размер данных: {data_tensor.shape}")
print(f"Размер меток: {labels_tensor.shape}")

Размер данных: torch.Size([12478, 3, 224, 224])
Размер меток: torch.Size([12478])


In [9]:
# Найти самое частое значение в labels_tensor
selected_label = labels_tensor.bincount().argmax().item()
print(f"Самая частая метка: {selected_label}")

Самая частая метка: 3


In [10]:
data_loader = DataLoader(TensorDataset(data_tensor, labels_tensor), batch_size=32, shuffle=True)

In [11]:
bin_label_tensor = (labels_tensor == selected_label).int()

In [12]:
torch.cuda.is_available()

False

In [ ]:
# Определение модели
model = models.resnet18(pretrained=True)  # Используем ResNet-18
model.fc = nn.Linear(model.fc.in_features, 1)  # Бинарная классификация


In [ ]:
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.resnet18(pretrained=True)  # Используем ResNet-18
model.fc = nn.Linear(model.fc.in_features, 1)  # Бинарная классификация
model = model.to("cuda" if torch.cuda.is_available() else "cpu")